
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
# pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')

import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args
from config_observer.CKPD import ckpd_to_CkpdObsConfig


cohort_config = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_config['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_config['ObsDTName'] = 'ObsDT'
cohort_config['PID_ObsDT_columns'] = [cohort_config['RootID'], cohort_config['ObsDTName']]
print(cohort_config)

g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace
{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_useb

# 1. DsCase

In [2]:
# 0. ************ RFT config ************
RecName_to_dsRec, RecName_to_dsRecInfo = {}, {}
cohort_label_list = [1, 2, 3]

# 1. ************ Case Trigger config ************
TriggerCaseMethod = 'CGM5MinEntry'


# 2. ************ InputCaseSetName ************
# option 1
# InputCaseSetName = 'C1.2.3-CGM5MinEntry'
InputCaseSetName = 'sftcgmbf24haf2h-rs42-ds0.1-out0.1tstail0.1vd0.1'
# InputCaseSetName = None 

In [3]:
from recfldtkn.loadtools import fetch_trigger_tools

Trigger_Tools = fetch_trigger_tools(TriggerCaseMethod, SPACE)
case_id_columns = Trigger_Tools['case_id_columns']
cohort_config['case_id_columns'] = case_id_columns
case_id_columns

['PID', 'ObsDT']

In [4]:
from recfldtkn.pipeline_case import get_ds_case_to_process

InputCaseSetName, df_case = get_ds_case_to_process(InputCaseSetName, 
                                                   cohort_label_list, 
                                                   TriggerCaseMethod, 
                                                   cohort_config, 
                                                   SPACE, 
                                                   RecName_to_dsRec, 
                                                   RecName_to_dsRecInfo)

logger.info(f'InputCaseSetName: {InputCaseSetName}')
logger.info(f'df_case shape: {df_case.shape}')

[INFO:2024-04-21 09:13:28,324:(269221877.py@11 __main__)]: InputCaseSetName: sftcgmbf24haf2h-rs42-ds0.1-out0.1tstail0.1vd0.1
[INFO:2024-04-21 09:13:28,324:(269221877.py@12 __main__)]: df_case shape: (1685533, 8)


In [5]:
df_case = df_case.sample(1000)
df_case.shape

(1000, 8)

# 2. Tagging

In [6]:
from recfldtkn.pipeline_case import process_df_tagging_tasks_in_chunks
from config_observer.QCF import cf_to_QueryCaseFeatConfig


CASE_TAGGING_PROC_CONFIG = {
    'use_CF_from_disk': False,
    'use_CO_from_disk': False,
    'start_chunk_id': 0,
    'end_chunk_id': None,
    'chunk_size': 500000,
    'save_to_pickle': False,
    'num_processors': 1
}

######################################
TagMethod_List = [
                'PttBasicWD', 
                
                'Bf24hCGMrn', 'Af2hCGMrn',
                'Bf24hCGMmode', 'Af2hCGMmode', 
                
                'Bf1mMEDALcf', 'Bf24hMEDALcf', 
                'Bf2hMEDALcf', 'Af2hMEDALcf',
                ]
######################################

if len(TagMethod_List) > 0:
    logger.info(f'df_case shape: {df_case.shape}')
    OutputCaseSetName, df_case = process_df_tagging_tasks_in_chunks(df_case, cohort_label_list, case_id_columns, 
                                                                    InputCaseSetName, 
                                                                    TagMethod_List, cf_to_QueryCaseFeatConfig, 
                                                                    cohort_config, SPACE, 
                                                                    RecName_to_dsRec, RecName_to_dsRecInfo,
                                                                    **CASE_TAGGING_PROC_CONFIG)
    logger.info(f'df_case shape: {df_case.shape}')


else:
    pass



[INFO:2024-04-21 09:13:29,395:(2735418505.py@28 __main__)]: df_case shape: (1000, 8)
[INFO:2024-04-21 09:13:29,395:(pipeline_case.py@420 recfldtkn.pipeline_case)]: chunk_id_list: range(0, 1)
[INFO:2024-04-21 09:13:29,650:(pipeline_case.py@308 recfldtkn.pipeline_case)]: ds_info is P: Dataset({
    features: ['PID', 'PatientID', 'YearOfBirth', 'ActivationDate', 'MRSegmentModifiedDateTime', 'UserTimeZone', 'UserTimeZoneOffset', 'Gender', 'MRSegmentID', 'DiseaseType'],
    num_rows: 11621
})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

[INFO:2024-04-21 09:14:17,808:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\3-RawData2023_CVSDeRxAug\ImpMed_data ...
[INFO:2024-04-21 09:14:17,962:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\1-RawData2022_CGM\LessonProg_data ...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[INFO:2024-04-21 09:14:23,176:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\3-RawData2023_CVSDeRxAug\ImpMed_data ...
[INFO:2024-04-21 09:14:23,395:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\1-RawData2022_CGM\LessonProg_data ...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[INFO:2024-04-21 09:14:27,960:(pipeline_case.py@462 recfldtkn.pipeline_case)]: chunk_id: 0, df_case_chunk_tagged: (1000, 69)
[INFO:2024-04-21 09:14:27,961:(2735418505.py@35 __main__)]: df_case shape: (1000, 69)


# Step 1: fPttBasicWD

In [7]:
df_case.columns

Index(['PID', 'ObsDT', 'name', 'Train', 'Valid', 'Test', 'In', 'Out', 'Gender',
       'YearOfBirth', 'DiseaseType', 'cohort', 'Age', 'Bf24hCGMrn:recnum',
       'Af2hCGMrn:recnum', 'Bf24hCGMmode:ModePercent',
       'Af2hCGMmode:ModePercent', 'Bf1mMedPres:recnum', 'Bf1mMedAdmin:recnum',
       'Bf1mImpMed:recnum', 'Bf1mCmt:recnum', 'Bf1mLesson:recnum',
       'Bf1mFood:recnum', 'Bf1mCarb:recnum', 'Bf1mExercise:recnum',
       'Bf1mSleep:recnum', 'Bf1mStep:recnum', 'Bf1mBP:recnum',
       'Bf1mWeigh:recnum', 'Bf1mLabTest:recnum', 'Bf24hMedPres:recnum',
       'Bf24hMedAdmin:recnum', 'Bf24hImpMed:recnum', 'Bf24hCmt:recnum',
       'Bf24hLesson:recnum', 'Bf24hFood:recnum', 'Bf24hCarb:recnum',
       'Bf24hExercise:recnum', 'Bf24hSleep:recnum', 'Bf24hStep:recnum',
       'Bf24hBP:recnum', 'Bf24hWeigh:recnum', 'Bf24hLabTest:recnum',
       'Bf2hMedPres:recnum', 'Bf2hMedAdmin:recnum', 'Bf2hImpMed:recnum',
       'Bf2hCmt:recnum', 'Bf2hLesson:recnum', 'Bf2hFood:recnum',
       'Bf2hCarb:recn

In [8]:
df_case['Gender'].value_counts()

Gender
2    549
1    450
3      1
Name: count, dtype: int64

In [9]:
##################################
FilterMethod = 'fPttBasicWD'
##################################
import inspect

def fn_case_filtering(df_case):
    # df_case = df_case[df_case['patient_gender'] != 'U'].reset_index(drop = True)
    df_case = df_case[df_case['Age'] >= 18].reset_index(drop = True)
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline\fn_learning\fPttBasicWD.py


In [10]:
# # Trigger2LearningCaseMethod
# pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
# module = load_module_variables(pypath)
# fn_case_filtering = module.fn_case_filtering

In [11]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

(1000, 69)
(988, 69)


# Step 2: fBf24h280CGM

In [12]:
df_case.columns

Index(['PID', 'ObsDT', 'name', 'Train', 'Valid', 'Test', 'In', 'Out', 'Gender',
       'YearOfBirth', 'DiseaseType', 'cohort', 'Age', 'Bf24hCGMrn:recnum',
       'Af2hCGMrn:recnum', 'Bf24hCGMmode:ModePercent',
       'Af2hCGMmode:ModePercent', 'Bf1mMedPres:recnum', 'Bf1mMedAdmin:recnum',
       'Bf1mImpMed:recnum', 'Bf1mCmt:recnum', 'Bf1mLesson:recnum',
       'Bf1mFood:recnum', 'Bf1mCarb:recnum', 'Bf1mExercise:recnum',
       'Bf1mSleep:recnum', 'Bf1mStep:recnum', 'Bf1mBP:recnum',
       'Bf1mWeigh:recnum', 'Bf1mLabTest:recnum', 'Bf24hMedPres:recnum',
       'Bf24hMedAdmin:recnum', 'Bf24hImpMed:recnum', 'Bf24hCmt:recnum',
       'Bf24hLesson:recnum', 'Bf24hFood:recnum', 'Bf24hCarb:recnum',
       'Bf24hExercise:recnum', 'Bf24hSleep:recnum', 'Bf24hStep:recnum',
       'Bf24hBP:recnum', 'Bf24hWeigh:recnum', 'Bf24hLabTest:recnum',
       'Bf2hMedPres:recnum', 'Bf2hMedAdmin:recnum', 'Bf2hImpMed:recnum',
       'Bf2hCmt:recnum', 'Bf2hLesson:recnum', 'Bf2hFood:recnum',
       'Bf2hCarb:recn

In [13]:
df_case['Bf24hCGMrn:recnum']

0      289.0
1      289.0
2      289.0
3      289.0
4      289.0
       ...  
983    289.0
984    289.0
985    289.0
986    289.0
987    289.0
Name: Bf24hCGMrn:recnum, Length: 988, dtype: float64

In [14]:
df_case['Bf24hCGMrn:recnum'].value_counts()

Bf24hCGMrn:recnum
289.0    888
284.0     11
288.0     10
277.0      9
278.0      8
283.0      8
279.0      8
285.0      8
281.0      7
282.0      7
286.0      5
280.0      5
276.0      4
287.0      4
274.0      4
275.0      2
Name: count, dtype: int64

In [15]:
##################################
FilterMethod = 'fBf24h280CGM'
##################################
import inspect

def fn_case_filtering(df_case):
    # df_case = df_case[df_case['patient_gender'] != 'U'].reset_index(drop = True)
    MinRecNum = 280
    
    if 'Bf24hCGMrn:recnum' in df_case.columns:
        df_case = df_case[df_case['Bf24hCGMrn:recnum'] >= MinRecNum].reset_index(drop = True)
    elif 'Bf24hCGMinfo:recnum' in df_case.columns:
        df_case = df_case[df_case['Bf24hCGMinfo:recnum'] >= MinRecNum].reset_index(drop = True)
    else:
        columns = list(df_case.columns)
        raise ValueError(f'No recnum column found: {columns}')
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline\fn_learning\fBf24h280CGM.py


In [16]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

(988, 69)
(953, 69)


# Step 2: fBf24h289CGM

In [17]:
##################################
FilterMethod = 'fBf24h289CGM'
##################################
import inspect

def fn_case_filtering(df_case):
    # df_case = df_case[df_case['patient_gender'] != 'U'].reset_index(drop = True)
    MinRecNum = 289
    if 'Bf24hCGMrn:recnum' in df_case.columns:
        df_case = df_case[df_case['Bf24hCGMrn:recnum'] >= MinRecNum].reset_index(drop = True)
    elif 'Bf24hCGMinfo:recnum' in df_case.columns:
        df_case = df_case[df_case['Bf24hCGMinfo:recnum'] >= MinRecNum].reset_index(drop = True)
    else:
        columns = list(df_case.columns)
        raise ValueError(f'No recnum column found: {columns}')
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline\fn_learning\fBf24h289CGM.py


In [18]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

(953, 69)
(888, 69)


# Step 3: fAf2h24CGM

In [19]:
##################################
FilterMethod = 'fAf2h24CGM'
##################################
import inspect

def fn_case_filtering(df_case):
    
    # df_case = df_case[df_case['patient_gender'] != 'U'].reset_index(drop = True)
    MinRecNum = 24
    if 'Af2hCGMrn:recnum' in df_case.columns:
        df_case = df_case[df_case['Af2hCGMrn:recnum'] >= MinRecNum].reset_index(drop = True)
    elif 'Af2hCGMinfo:recnum' in df_case.columns:
        df_case = df_case[df_case['Af2hCGMinfo:recnum'] >= MinRecNum].reset_index(drop = True)
    else:
        columns = list(df_case.columns)
        raise ValueError(f'No recnum column found: {columns}')
    
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline\fn_learning\fAf2h24CGM.py


In [20]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

(888, 69)
(888, 69)


## Step 4: fBf24HModePctn40

In [21]:
##################################
FilterMethod = 'fBf24HModePctn40'
##################################
import inspect

def fn_case_filtering(df_case):
   
    MaxModePercent = 0.4
    if 'Bf24hCGMinfo:ModePercent' in df_case.columns:
        df_case = df_case[df_case['Bf24hCGMinfo:ModePercent'] <= MaxModePercent].reset_index(drop = True)
    elif 'Bf24hCGMmode:ModePercent' in df_case.columns:
        df_case = df_case[df_case['Bf24hCGMmode:ModePercent'] <= MaxModePercent].reset_index(drop = True)
    else:
        columns = list(df_case.columns)
        raise ValueError(f'No recnum column found: {columns}')
    
    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline\fn_learning\fBf24HModePctn40.py


In [22]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

(888, 69)
(887, 69)


## Step 5: fAf2HModePctn40

In [23]:
##################################
FilterMethod = 'fAf2HModePctn40'
##################################
import inspect

def fn_case_filtering(df_case):
    # df_case = df_case[df_case['patient_gender'] != 'U'].reset_index(drop = True)
    # df_case = df_case[df_case['Af2hCGMmode:ModePercent'] <= 0.4].reset_index(drop = True)
    
    MaxModePercent = 0.4
    if 'Af2hCGMmode:ModePercent' in df_case.columns:
        df_case = df_case[df_case['Af2hCGMmode:ModePercent'] <= MaxModePercent].reset_index(drop = True)
    elif 'Af2hCGMinfo:ModePercent' in df_case.columns:
        df_case = df_case[df_case['Af2hCGMinfo:ModePercent'] <= MaxModePercent].reset_index(drop = True)
    else:
        columns = list(df_case.columns)
        raise ValueError(f'No recnum column found: {columns}')
    

    return df_case

fn_case_filtering.fn_string = inspect.getsource(fn_case_filtering)

from recfldtkn.loadtools import convert_variables_to_pystirng

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [fn_case_filtering]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{FilterMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline\fn_learning\fAf2HModePctn40.py


In [24]:
print(df_case.shape)
df_case = fn_case_filtering(df_case)
print(df_case.shape)

(887, 69)
(879, 69)
